<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv


--2020-03-04 16:28:45--  https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24394136 (23M) [text/plain]
Saving to: ‘small_data.csv.3’

small_data.csv.3    100%[===================>]  23.26M   115MB/s    in 0.2s    

2020-03-04 16:28:46 (115 MB/s) - ‘small_data.csv.3’ saved [24394136/24394136]



In [0]:
import pandas as pd
from sklearn import cluster
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation, MeanShift, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD, NMF

In [0]:

from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn.datasets import make_blobs

In [0]:
data = pd.read_csv('small_data.csv')

In [5]:
data.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,793dd3d5f0f0,9467e0341c0e,Алтайский край,Барнаул,Личные вещи,Детская одежда и обувь,Для девочек,Платья и юбки,146-152 см (10-12 лет),Платье праздничное для девочки,Очень красивое торг,1500.0,106,2017-04-12,Company,ace7b2b7f066a6b6ff244f90d030c2c1a1bf118dd0e1ac...,573.0
1,f8c965d9b668,1905516660f5,Свердловская область,Каменск-Уральский,Для дома и дачи,Мебель и интерьер,"Кровати, диваны и кресла",NaN,NaN,Кровать двухспальная,Двуспальная кровать 140х200 с металлическим ка...,5700.0,10,2017-04-15,Private,b38ef24dd5b876ed626bd98241e228e4e19e4160d33abe...,1439.0
2,2f153c00d561,b4dc4849fdd4,Самарская область,Самара,Бытовая электроника,Телефоны,Samsung,NaN,NaN,Samsung S3 mini gt-i8190,"Продам телефон самсунг 2 ядра, 1 гиг оперативн...",1500.0,6,2017-04-12,Private,8530cc0c8d5dc6d212497dffd610d89b92a7db952934db...,2932.0
3,7fc5eea56915,02c14d39fdb4,Саратовская область,Саратов,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,44–46 (M),"Красивое платье на одно плечо, в идеале размер м","Заказывала в Сша, прекрасное качество, в деле ...",1000.0,1280,2017-04-13,Company,eb21745ba276d3b00b05856a58fe719e31a82419c0221a...,386.0
4,b01000d29ea3,b17f14110367,Удмуртия,Ижевск,Личные вещи,Детская одежда и обувь,Для мальчиков,"Шапки, варежки, шарфы",NaN,Стильная панама,Состояние новой. Размер до года- полтора. По ...,150.0,145,2017-04-14,Company,5bf1001fa681b96f121ad8047c85e3747f3dac37830dd0...,209.0


In [0]:
data = data[['category_name', 'title']]

In [7]:
data.shape

(37339, 2)

In [0]:
sample = data.sample(frac=0.1)

In [111]:
sample.shape

(3734, 2)

In [0]:

cv = CountVectorizer(max_features=500)
X = cv.fit_transform(sample['title'])

y = sample['category_name']

In [113]:
cluster = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster.fit(X)

Converged after 272 iterations.


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
                    damping=0.7, max_iter=400, preference=-5, verbose=2)

In [0]:
labels = cluster.labels_

In [115]:
n_clusters = len(set(labels))
n_clusters

224

In [116]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Silhouette Coefficient: 0.360


In [117]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.497
Completeness: 0.380
V-measure: 0.431
Adjusted Rand Index: -0.007
Adjusted Mutual Information: 0.303


In [0]:
sample['cluster'] = cluster.labels_

In [119]:
sample[sample.cluster==39].head(10)

,category_name,title,cluster
27691,Комнаты,"Комната 29.6 м² в 2-к, 1/2 эт.",39
22099,Комнаты,"Комната 13 м² в 1-к, 5/5 эт.",39
5473,Комнаты,"Комната 13 м² в 4-к, 4/5 эт.",39
18523,Комнаты,"Комната 13 м² в 1-к, 5/5 эт.",39
19147,Комнаты,"Комната 13 м² в 1-к, 4/5 эт.",39


In [120]:
sample[sample.category_name=='Телефоны'].cluster.value_counts()

0      43
213    23
90      8
34      8
112     8
107     7
68      7
73      6
158     5
2       5
176     5
62      4
19      4
5       3
193     2
100     1
96      1
118     1
140     1
7       1
Name: cluster, dtype: int64

In [0]:
#MS тоже не умеет работать с sparse матрицами

In [0]:
cv = CountVectorizer(max_features=2000)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [196]:
cluster.estimate_bandwidth(X_svd)

AttributeError: ignored

In [197]:
# cluster_all=False чтобы не записывать все объекты в кластеры
cluster = MeanShift(cluster_all=False, bandwidth=0.5)
cluster.fit(X_svd)

MeanShift(bandwidth=0.5, bin_seeding=False, cluster_all=False, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [0]:
sample = data.sample(frac=0.5)

In [199]:
sample.shape

(18670, 3)

In [0]:

cv = CountVectorizer(max_features=500)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [201]:
len(y.unique())

47

In [202]:
X.shape

(18670, 500)

In [203]:
import sklearn
sklearn.cluster.estimate_bandwidth(X_svd)

0.7140792571536527

In [204]:
# cluster_all=False чтобы не записывать все объекты в кластеры
cluster = MeanShift(cluster_all=False, bandwidth=0.5)
cluster.fit(X_svd)

KeyboardInterrupt: ignored

In [0]:
len(set(cluster.labels_))

In [0]:
labels = cluster.labels_

In [0]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [0]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

In [0]:
sample['cluster'] = cluster.labels_

In [0]:
sample[sample.cluster==2].head()

In [0]:
sample = data.sample(frac=0.01)

In [0]:
sample.shape

In [0]:
cv = CountVectorizer(max_features=500)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [0]:
cluster = SpectralClustering(n_clusters=47)
cluster.fit(X_svd)

In [0]:
X.shape

In [0]:
labels = cluster.labels_
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [0]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

In [0]:
sample['cluster'] = cluster.labels_

In [0]:
sample[sample.cluster==17]

In [0]:
sample = data.sample(frac=0.5)

In [0]:
sample.shape

In [0]:
cv = CountVectorizer(max_features=2000)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [0]:
X.shape

In [0]:

# eps - задает максимальную для объединения дистанцию между объектами == размер окрестности
# min_samples - минимальное количество объектов в классе

cluster = DBSCAN(min_samples=10, eps=0.3) 
cluster.fit(X_svd)

In [0]:
sample['cluster'] = cluster.labels_

In [0]:
sample[sample.cluster==1].head()

In [0]:
sample = data

In [0]:
sample.shape

In [0]:
cv = CountVectorizer(max_features=500)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [0]:
X.shape

In [0]:

cluster = KMeans(n_clusters=47)
cluster.fit(X_svd)

In [0]:
labels = cluster.labels_

In [0]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))

In [0]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

In [0]:
sample['cluster'] = cluster.labels_
sample[sample.cluster==13].head()

In [0]:
#MiniBatchKMeans

In [0]:
cv = CountVectorizer(max_features=5000)
svd = TruncatedSVD(200)
X = cv.fit_transform(data['title'])
X_svd = svd.fit_transform(X)

y = data['category_name']

In [0]:
X.shape

In [0]:
cluster = MiniBatchKMeans(n_clusters=1000, init_size=5000, verbose=1, max_iter=5000, 
                          max_no_improvement=100, reassignment_ratio=0.3)
cluster.fit(X_svd)

In [0]:

labels = cluster.labels_

In [0]:

print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))

In [0]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

In [0]:
sample['cluster'] = cluster.labels_
sample[sample.cluster==13].head()

In [0]:
########################

In [0]:
def cluster_metrics(cluster, sample, cluster_num, X, y):
  labels = cluster.labels_

  print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))
  
  # эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
  print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
  print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

  print("Adjusted Rand Index: %0.3f"
        % metrics.adjusted_rand_score(y, labels))
  print("Adjusted Mutual Information: %0.3f"
        % metrics.adjusted_mutual_info_score(y, labels))
  
  sample['cluster'] = cluster.labels_
  print(sample[sample.cluster==cluster_num].head(7)) 


In [0]:
sample1 = data.sample(frac=0.01)


In [0]:
cv = CountVectorizer(max_features=500)
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

In [0]:
tfidf = TfidfVectorizer(max_features=500)
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']


In [62]:
cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_

Converged after 173 iterations.


In [63]:
cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

Converged after 15 iterations.


In [64]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.060
Homogeneity: 0.229
Completeness: 0.497
V-measure: 0.313
Adjusted Rand Index: -0.021
Adjusted Mutual Information: 0.138
          category_name                                  title  cluster
7016  Мебель и интерьер  84 Диван книжка - Эко кожа.От фабрики       11


In [65]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: -0.021
Homogeneity: 0.192
Completeness: 0.227
V-measure: 0.208
Adjusted Rand Index: 0.004
Adjusted Mutual Information: -0.002
                    category_name                              title  cluster
36516   Одежда, обувь, аксессуары      Блузка с шифоновыми вставками       11
7331    Одежда, обувь, аксессуары                              Горка       11
14041      Детская одежда и обувь                             Платье       11
3330           Коллекционирование  Самовар электрический новый 99г/в       11
23524  Товары для детей и игрушки                      Пелёнка-кокон       11
27494        Дома, дачи, коттеджи      Дом 186 м² на участке 16 сот.       11
27766      Детская одежда и обувь            Кроссовки с динозаврами       11


In [0]:
sample = data.sample(frac=0.01)
cv = CountVectorizer(max_features=500)
tfidf = TfidfVectorizer(max_features=500)
svd = TruncatedSVD(50)
nmf = NMF(50)
X_cv = cv.fit_transform(sample['title'])
X_cv = tfidf.fit_transform(sample['title'])
y_cv = sample1['category_name']

X_svd_cv = svd.fit_transform(X_cv)
X_svd_tf = svd.fit_transform(X_tf)

X_nmf_cv = svd.fit_transform(X_cv)
X_nmf_tf = svd.fit_transform(X_tf)


y = sample['category_name']

In [72]:
# cluster2 = AgglomerativeClustering(n_clusters=170)
# cluster2_svd_cv.fit(X_svd_cv)

cluster2_svd_cv = AgglomerativeClustering(n_clusters=170)
cluster2_svd_cv.fit(X_svd_cv)

cluster2_svd_tf = AgglomerativeClustering(n_clusters=170)
cluster2_svd_tf.fit(X_svd_tf)

cluster2_nmf_cv = AgglomerativeClustering(n_clusters=170)
cluster2_nmf_cv.fit(X_nmf_cv)

cluster2_nmf_tf = AgglomerativeClustering(n_clusters=170)
cluster2_nmf_tf.fit(X_nmf_tf)


AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=170)

In [73]:
cluster_metrics(cluster2_svd_cv, sample1, 11, X_svd_cv, y_cv)

Silhouette Coefficient: 0.385
Homogeneity: 0.633
Completeness: 0.436
V-measure: 0.516
Adjusted Rand Index: -0.005
Adjusted Mutual Information: 0.004
                 category_name             title  cluster
20457     Гаражи и машиноместа       Куплю гараж       11
11539  Грузовики и спецтехника             Krone       11
14093               Автомобили  Opel Astra, 2010       11


In [74]:
cluster_metrics(cluster2_svd_tf, sample1, 11, X_svd_tf, y_cv)

Silhouette Coefficient: 0.377
Homogeneity: 0.717
Completeness: 0.469
V-measure: 0.567
Adjusted Rand Index: 0.018
Adjusted Mutual Information: 0.093
                   category_name                     title  cluster
6457   Одежда, обувь, аксессуары  Ярко-синее летнее платье       11
13936     Детская одежда и обувь             Летнее платье       11


In [75]:
cluster_metrics(cluster2_nmf_cv, sample1, 11, X_nmf_cv, y_cv)

Silhouette Coefficient: 0.371
Homogeneity: 0.630
Completeness: 0.432
V-measure: 0.512
Adjusted Rand Index: -0.005
Adjusted Mutual Information: -0.004
                 category_name             title  cluster
20457     Гаражи и машиноместа       Куплю гараж       11
11539  Грузовики и спецтехника             Krone       11
14093               Автомобили  Opel Astra, 2010       11


In [76]:
cluster_metrics(cluster2_nmf_tf, sample1, 11, X_nmf_tf, y_cv)

Silhouette Coefficient: 0.382
Homogeneity: 0.719
Completeness: 0.470
V-measure: 0.569
Adjusted Rand Index: 0.021
Adjusted Mutual Information: 0.098
                    category_name  ... cluster
16754  Товары для детей и игрушки  ...      11
624                      Телефоны  ...      11
20435           Предложение услуг  ...      11
4092                   Автомобили  ...      11
15898          Красота и здоровье  ...      11
18153                 Фототехника  ...      11
8403           Красота и здоровье  ...      11

[7 rows x 3 columns]
